In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import sys
import os
from scipy import stats
sys.path.insert(0, os.path.abspath('..'))

In [2]:
%load_ext autoreload
%autoreload 2
import evidently

/home/eoin/miniconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.kde module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
from ipywidgets import interact, FloatSlider
def fs(v, low, high, step, desc=''):
    return FloatSlider(value=v, min=low, max=high, step=step, description=desc, continuous_update=False)


# Simple Wald Model


In [ ]:
print(evidently.Wald.__doc__)

In [ ]:
def wald_simulation_plot(t0=1., v=.5, z=0, a=1., c=1.):
    model = evidently.Wald(pars=[t0, v, z, a, c], max_time=5)
    evidently.viz.visualise_ddm(model, model_type='wald', measure='raw')
    title = 'Start time = %.1f, Drift = %.1f, Start = %.1f, Threshold = %.1f' % (t0, v, z, a)
    plt.suptitle(title, y=1.01)

interact(wald_simulation_plot,
         t0  = fs(1., 0, 5., .1,   'Start time'),
         v   = fs(.5, 0, 2., .1,   'Drift'),
         z   = fs(0, 0, .5, .1,   'Starting point'),
         a   = fs(.5, 0., 2., .1,   'Threshold'),
         c   = fs(.1, 0., 2., .1,   'Noise SD'));

# Simple Drift Diffusion Model


In [ ]:
print(evidently.Diffusion.__doc__)

In [ ]:
def ddm_simulation_plot(t0=1., v=.5, z=0., a=.5, c=.1):
    model = evidently.Diffusion(pars=[t0, v, z, a, c])
    evidently.viz.visualise_ddm(model)
    title = 't0 = %.1f, Drift = %.1f, Bias = %.1f, Threshold = %.1f' % (t0, v, z, a)
    plt.suptitle(title, y=1.01)

interact(ddm_simulation_plot,
         t0  = fs(1., 0, 2., .1,   't0'),
         v   = fs(.5, 0, 2., .1,   'Drift'),
         z   = fs(0., -1., 1., .1,  'Bias'),
         a     = fs(.5, 0., 2., .1,   'Threshold'),
         c   = fs(.1, 0., 1., .1,   'Noise SD'));

# Hierarchical Drift Diffusion Model

In [ ]:
print(evidently.HDiffusion.__doc__)

In [ ]:
def hddm_simulation_plot(m_t0=1., m_v=.5, m_z=0., m_a=.5,
                         s_t0=.1, s_v=.1, s_z=.1, s_a=.1, c=.1):
    model = evidently.HDiffusion(pars=[m_t0, m_v, m_z, m_a, s_t0, s_v, s_z, s_a, c])
    evidently.viz.visualise_ddm(model, measure='means', threshold=m_a)
    title = 't0 = %.1f (%.1f), Drift = %.1f (%.1f), Bias = %.1f (%.1f), Threshold = %.1f (%.1f), Noise = %.1f' % (
        m_t0, s_t0, m_v, s_v, m_z, s_z, m_a, s_a, c)
    plt.suptitle(title, y=1.05)

interact(hddm_simulation_plot,
    m_t0 = fs(1., 0., 1., .1, 'T0'),
    m_v  = fs(.5, -1., 1., .1, 'Drift'),
    m_z  = fs(0., -1., 1., .1, 'Bias'),
    m_a  = fs(1., 0., 2., .1, 'Threshold'),
    s_t0 = fs(.1, 0., 1., .1, 'SD T0'),
    s_v  = fs(.1, 0., 1., .1, 'SD Drift'),
    s_z  = fs(.1, 0., 1., .1, 'SD Bias'),
    s_a  = fs(.1, 0., 1., .1, 'SD Threshold'),
    c    = fs(.1, 0., 1., .1, 'Noise'));


# Race

In [4]:
print(evidently.Race.__doc__)

The classic Race model.

    This model has nine parameters (!), where x1 is accumulator #1, x2
    is accumulator #2:

    - t1: Onset of evidence accumulation (seconds) for x1
    - v1: Drift rate for x1
    - z1: Starting point for x1
    - c1: Noise for x1
    - t2: Onset of evidence accumulation (seconds) for x2
    - v2: Drift rate for x2
    - z2: Starting point for x2
    - c2: Noise for x2
    - a: Threshold.

    The model equation for each accumulator is

        | x_{t+1} = x_{t} + dt * input_{t} + sqrt(dt) * ε;
        | x_{0} = z * a
        | ε ~ Normal(0, c);
        | input_{t} = where(time > t, v, 0)

    The first accumulator to cross threshold, a, produces a response.
    Both accumulators use the same threshold, but can differ in their
    starting points. It is suggested that you keep `a=1` and vary the
    other parameters.

    Notes
    =====

    This model is extremely overparameterised, and cannot be fit to
    real data without fixing the value of several p

In [30]:
model = evidently.Race(pars=[2.1, 2, 0., .2, 
                             2.2, 2, .2, .1, 1.])

In [ ]:
evidently.viz.setup_race_plot(model)
evidently.viz._plot_race_results(X1.values, X2.values, r, t)

In [39]:
def ddm_simulation_plot(t1, v1, z1, c1, t2, v2, z2, c2, a):
    model = evidently.Race(pars=[t1, v1, z1, c1, t2, v2, z2, c2, a])
    evidently.viz.visualise_model(model, model_type='race', measure='means')
    title = 'Option #1: t0 = %.1f, Drift = %.1f, Bias = %.1f, Noise = %.1f' % (t1, v1, z1, c1)
    title += '\nOption #2: t0 = %.1f, Drift = %.1f, Bias = %.1f, Noise = %.1f' % (t2, v2, z2, c2)
    title += '\nThreshold: %.1f' % a    
    plt.suptitle(title, y=1.2)

interact(ddm_simulation_plot,
         t1   = fs(1., 0, 2., .1,   '#1: t0'),
         v1   = fs(.5, 0, 2., .1,   '#1: Drift'),
         z1   = fs(0., 0, 1., .1,  '#1: Bias'),
         c1   = fs(.1, 0., 1., .1,   '#1: Noise SD'),
         t2   = fs(1., 0, 2., .1,   '#2: t0'),
         v2   = fs(.5, 0, 2., .1,   '#2: Drift'),
         z2   = fs(0., 0., 1., .1,  '#2: Bias'),
         c2   = fs(.1, 0., 1., .1,   '#2: Noise SD'),
         a     = fs(.5, 0., 2., .1,  'Threshold'));

interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='#1: t0', max=2.0), FloatSli…